In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"#指定在第0块GPU上跑
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)
config= tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction =1  #占用40%显存

batch_size=100

n_batch=mnist.train.num_examples//batch_size

#初始化权值

def weight_variables(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variables(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

x_image=tf.reshape(x,[-1,28,28,1])

W_conv1=weight_variables([5,5,1,8])
b_conv1=bias_variables([8])

h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)

W_conv2=weight_variables([5,5,8,16])
b_conv2=bias_variables([16])

h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)


W_fc1=weight_variables([7*7*16,200])
b_fc1=bias_variables([200])

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*16])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)


keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

W_fc2=weight_variables([200,10])
b_fc2=bias_variables([10])

predicction=tf.nn.softmax(tf.matmul(h_fc1,W_fc2)+b_fc2)

cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=predicction))

train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(predicction,1))

accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print('Iter '+str(epoch)+',Testing Accuracy= '+str(acc))



W0725 00:37:27.447940  6668 deprecation.py:323] From <ipython-input-2-c8db7cb23bf8>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 00:37:27.457940  6668 deprecation.py:323] From C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 00:37:27.462940  6668 deprecation.py:323] From C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0725 00:37:28.109977  6668 deprecation.py:323] From C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0725 00:37:28.116978  6668 deprecation.py:323] From C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0725 00:37:28.226984  6668 deprecation.py:323] From C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.p

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


W0725 00:37:28.870021  6668 deprecation.py:506] From <ipython-input-2-c8db7cb23bf8>:53: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0725 00:37:28.959026  6668 deprecation.py:323] From <ipython-input-2-c8db7cb23bf8>:60: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Iter 0,Testing Accuracy= 0.7504


KeyboardInterrupt: 